In [2]:
from ultralytics import RTDETR

# Load a COCO-pretrained RT-DETR-l model
model = RTDETR("rtdetr-l.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/ec2-user/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 63.4M/63.4M [00:00<00:00, 403MB/s]


In [1]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
#!/usr/bin/env python3
"""
Test script to demonstrate how to use the RF-DETR object detection API locally.
This version is specifically optimized for testing against a server already running with the serve script.

Usage:
    1. Command line: python test_local.py --image test_image.jpg [--visualize] [--confidence 0.25]
    2. Direct call: main(image_path="test_image.jpg", visualize=True, confidence=0.25, url="http://localhost:8080")
"""

import argparse
import requests
import json
import base64
import sys
from pathlib import Path

def encode_image_to_base64(image_path):
    """Encode an image file to base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def main(image_path=None, visualize=False, confidence=0.25, url="http://localhost:8080"):
    """
    Test the RF-DETR object detection API.
    
    Args:
        image_path (str): Path to the input image
        visualize (bool): Whether to visualize the detection results
        confidence (float): Confidence threshold for detections
        url (str): Base URL of the API
        
    Returns:
        dict: Detection results from the API
    """
    # Parse command line args if not provided as function parameters
    if image_path is None:
        parser = argparse.ArgumentParser(description='Test RF-DETR object detection API against serve deployment.')
        parser.add_argument('--image', type=str, required=True, help='Path to the image file')
        parser.add_argument('--visualize', action='store_true', help='Visualize detection results')
        parser.add_argument('--confidence', type=float, default=0.25, help='Confidence threshold (0.0-1.0)')
        parser.add_argument('--url', type=str, default='http://localhost:8080', 
                            help='Base URL of the API')
        args = parser.parse_args()
        
        image_path = args.image
        visualize = args.visualize
        confidence = args.confidence
        url = args.url
    
    # Convert to Path object
    image_path = Path(image_path)
    
    # Check if image exists
    if not image_path.exists():
        print(f"Error: Image not found at {image_path}")
        return None
        
    # Print info
    print(f"Testing RF-DETR object detection with:")
    print(f"  Image: {image_path}")
    print(f"  Confidence threshold: {confidence}")
    print(f"  Visualize: {visualize}")
    print(f"  API URL: {url}")
    print("-" * 40)
    
    # Check if the server is ready
    try:
        response = requests.get(f"{url}/ping", timeout=5)
        if response.status_code == 200:
            print("Server is ready!")
        else:
            print(f"Warning: Server health check returned status {response.status_code}")
    except requests.RequestException as e:
        print(f"Warning: Could not connect to server: {e}")
        print("Continuing with inference request anyway...")
    
    # Send the inference request to the invocations endpoint (SageMaker compatible)
    try:
        # Choose between binary and JSON format
        if visualize:
            # Send binary image and get binary image back
            with open(image_path, 'rb') as f:
                image_data = f.read()
            
            print("Sending binary request to /invocations endpoint...")
            response = requests.post(
                f"{url}/invocations",
                data=image_data,
                headers={
                    "Content-Type": "application/octet-stream",
                    "Accept": "application/octet-stream"
                }
            )
            
        else:
            # Send JSON and get JSON back
            image_b64 = encode_image_to_base64(image_path)
            
            payload = {
                "image": image_b64,
                "confidence_threshold": confidence,
                "visualize": visualize
            }
            
            print("Sending JSON request to /invocations endpoint...")
            response = requests.post(
                f"{url}/invocations",
                json=payload,
                headers={
                    "Content-Type": "application/json",
                    "Accept": "application/json"
                }
            )
            
    except Exception as e:
        print(f"Error during inference request: {e}")
        return None

    # Process the response
    result = None
    if response.status_code == 200:
        print(f"Success! Status code: {response.status_code}")
        
        # Check the content type
        content_type = response.headers.get('Content-Type', '')
        
        if 'image' in content_type or ('octet-stream' in content_type and visualize):
            # Save the image response
            output_path = f"output_{image_path.stem}.jpg"
            with open(output_path, 'wb') as f:
                f.write(response.content)
            print(f"Detection visualization saved to {output_path}")
            result = {"output_image": output_path}
            
        else:
            # Process the JSON response
            try:
                result = response.json()
                print("\nDetection Results:")
                print(f"Inference time: {result.get('inference_time', 'N/A')}s")
                
                detections = result.get('detections', [])
                print(f"Found {len(detections)} object(s)")
                
                for i, det in enumerate(detections, 1):
                    print(f"\nObject {i}:")
                    print(f"  Class: {det.get('class', 'unknown')}")
                    print(f"  Confidence: {det.get('confidence', 0):.4f}")
                    box = det.get('box', {})
                    print(f"  Bounding Box: [{box.get('x1', 0):.1f}, {box.get('y1', 0):.1f}] to [{box.get('x2', 0):.1f}, {box.get('y2', 0):.1f}]")
                
                # If there's a base64 image in the response, save it
                if visualize and 'image' in result:
                    try:
                        image_data = base64.b64decode(result['image'])
                        output_path = f"output_{image_path.stem}.jpg"
                        with open(output_path, 'wb') as f:
                            f.write(image_data)
                        print(f"\nDetection visualization saved to {output_path}")
                        result["output_image"] = output_path
                    except Exception as e:
                        print(f"Error saving visualization: {e}")
                
            except Exception as e:
                print(f"Error parsing JSON response: {e}")
                print("Raw response:", response.text[:1000])  # Print first 1000 chars
                result = {"error": str(e), "raw_response": response.text[:1000]}
    else:
        print(f"Error: Status code {response.status_code}")
        print("Response:", response.text)
        result = {"error": f"Status code {response.status_code}", "response": response.text}
        
    return result

# Example of how to use this script directly by importing it
if __name__ == "__main__":
    
    # Example 1: Basic detection with default confidence
    result = main(
        image_path="id.png",  # Replace with actual image path
        visualize=False,
        confidence=0.25,
        url="http://localhost:8080"
    )
    
    print("\nExample complete. Result summary:")
    if result:
        if "output_image" in result:
            print(f"Image saved to: {result['output_image']}")
        elif "detections" in result:
            print(f"Detected {len(result['detections'])} objects")
    else:
        print("No result returned")

Testing RF-DETR object detection with:
  Image: id.png
  Confidence threshold: 0.25
  Visualize: False
  API URL: http://localhost:8080
----------------------------------------
Server is ready!
Sending JSON request to /invocations endpoint...
Success! Status code: 200

Detection Results:
Inference time: 0.041s
Found 3 object(s)

Object 1:
  Class: person
  Confidence: 0.9489
  Bounding Box: [33.4, 0.7] to [385.4, 288.6]

Object 2:
  Class: tie
  Confidence: 0.5082
  Bounding Box: [152.3, 123.9] to [219.5, 288.1]

Object 3:
  Class: book
  Confidence: 0.2724
  Bounding Box: [-0.1, 1.2] to [29.4, 53.3]

Example complete. Result summary:
Detected 3 objects


In [5]:
result

{'detections': [{'box': {'x1': 546.64,
    'y1': 398.24,
    'x2': 3310.63,
    'y2': 2228.56},
   'confidence': 0.9771,
   'class_id': 5,
   'class': 'bus'},
  {'box': {'x1': 2294.88, 'y1': 1344.61, 'x2': 2494.93, 'y2': 1574.7},
   'confidence': 0.6957,
   'class_id': 0,
   'class': 'person'},
  {'box': {'x1': 0.02, 'y1': 1915.47, 'x2': 52.52, 'y2': 2390.83},
   'confidence': 0.3016,
   'class_id': 2,
   'class': 'car'}],
 'inference_time': 2.4554,
 'image_width': 3874,
 'image_height': 2583}

In [31]:
%%sh

# Specify an algorithm name
algorithm_name=rf-detr-inference

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



Login Succeeded


#0 building with "default" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 1.28kB done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/pytorch/pytorch:1.12.1-cuda11.3-cudnn8-runtime
#2 DONE 0.1s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [ 1/10] FROM docker.io/pytorch/pytorch:1.12.1-cuda11.3-cudnn8-runtime@sha256:0bc0971dc8ae319af610d493aced87df46255c9508a8b9e9bc365f11a56e7b75
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 137B done
#5 DONE 0.0s

#6 [ 8/10] WORKDIR /opt/ml/code
#6 CACHED

#7 [ 6/10] COPY sagemaker_serve.py /opt/ml/code/
#7 CACHED

#8 [ 5/10] COPY rf_detr_detector.py /opt/ml/code/
#8 CACHED

#9 [ 3/10] RUN pip install --no-cache-dir     fastapi     uvicorn     gunicorn     ultralytics     opencv-python     pillow     numpy     python-multipart
#9 CACHED

#10 [ 4/10] RUN mkdir -p /opt/ml/code /opt/ml/model
#10 CACHED

#11 [ 9/10] COPY se

The push refers to repository [707684582322.dkr.ecr.us-east-1.amazonaws.com/rf-detr-inference]
6172ed331c89: Preparing
1448aeddd1da: Preparing
5f70bf18a086: Preparing
a66e647d4543: Preparing
826251ad4c60: Preparing
cd064b8848b1: Preparing
076c1f2d33a5: Preparing
b142358ca16b: Preparing
1e088a044afa: Preparing
d400ef3df394: Preparing
14ff0720512e: Preparing
907222238196: Preparing
b142358ca16b: Waiting
cd064b8848b1: Waiting
1e088a044afa: Waiting
e722d396f503: Preparing
d400ef3df394: Waiting
907222238196: Waiting
e722d396f503: Waiting
14ff0720512e: Waiting
076c1f2d33a5: Waiting
5f70bf18a086: Layer already exists
a66e647d4543: Layer already exists
6172ed331c89: Layer already exists
826251ad4c60: Layer already exists
1448aeddd1da: Layer already exists
cd064b8848b1: Layer already exists
076c1f2d33a5: Layer already exists
b142358ca16b: Layer already exists
1e088a044afa: Layer already exists
d400ef3df394: Layer already exists
14ff0720512e: Layer already exists
907222238196: Layer already exis

In [32]:
import boto3
import json
import time

# Set your ECR image URI
image_uri = "707684582322.dkr.ecr.us-east-1.amazonaws.com/rf-detr-inference"

In [33]:
# Initialize boto3 clients
session = boto3.Session()
sm_client = session.client('sagemaker')

# Get the SageMaker execution role from the notebook environment
import sagemaker
role_arn = sagemaker.get_execution_role()
print(f"Using SageMaker execution role: {role_arn}")

[05/12/25 06:14:52] INFO     Found credentials from IAM Role:                                   ]8;id=937732;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=879297;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=488802;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=214380;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using SageMaker execution role: arn:aws:iam::707684582322:role/service-role/AmazonSageMaker-ExecutionRole-20191024T163188


In [35]:
model_name = "rtdetr"
instance_type = "ml.g5.2xlarge"

In [36]:
    # Create model
    print(f"Creating model: {model_name}")
    model_response = sm_client.create_model(
        ModelName=model_name,
        PrimaryContainer={
            'Image': image_uri,
            'Mode': 'SingleModel',
        },
        ExecutionRoleArn=role_arn
    )
    print(f"Model ARN: {model_response['ModelArn']}")

Creating model: rtdetr
Model ARN: arn:aws:sagemaker:us-east-1:707684582322:model/rtdetr


In [37]:
# Create endpoint configuration
print(f"Creating endpoint configuration: {model_name}-config")
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=f"{model_name}-config",
    ProductionVariants=[
        {
            'VariantName': 'AllTraffic',
            'ModelName': model_name,
            'InstanceType': instance_type,
            'InitialInstanceCount': 1,
        }
    ]
)
print(f"Endpoint config ARN: {endpoint_config_response['EndpointConfigArn']}")

Creating endpoint configuration: rtdetr-config
Endpoint config ARN: arn:aws:sagemaker:us-east-1:707684582322:endpoint-config/rtdetr-config


In [49]:
endpoint_name = "deployment-rtdetr-gpu"

In [50]:
# Create endpoint
print(f"Creating endpoint: {endpoint_name}")
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=f"{model_name}-config"
)
print(f"Endpoint ARN: {endpoint_response['EndpointArn']}")

# Wait for endpoint to be in service
print("Waiting for endpoint to be in service...")

Creating endpoint: deployment-rtdetr-gpu


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 # Create endpoint                                                                           │
│    2 print(f"Creating endpoint: {endpoint_name}")                                                │
│ ❱  3 endpoint_response = sm_client.create_endpoint(                                              │
│    4 │   EndpointName=endpoint_name,                                                             │
│    5 │   EndpointConfigName=f"{model_name}-config"                                               │
│    6 )                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/client.py:570   │
│ in _api_call                                                                                     │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/context.py:123  │
│ in wrapper                                                                                       │
│                                                                                                  │
│   120 │   │   │   with start_as_current_context():                                               │
│   121 │   │   │   │   if hook:                                                                   │
│   122 │   │   │   │   │   hook()                                                                 │
│ ❱ 123 │   │   │   │   return func(*args, **kwargs)                                               │
│   124 │   │                                                                                      │
│   125 │   │   return wrapper                                                                     │
│   126                                                                                            │
│                                                                                                  │
│ /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/client.py:1031  │
│ in _make_api_call                                                                                │
│                                                                                                  │
│   1028 │   │   │   │   "Code"                                                                    │
│   1029 │   │   │   )                                                                             │
│   1030 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1031 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1032 │   │   else:                                       

In [51]:
import base64
image_path= "./id.png"

def encode_image_to_base64(image_path):
    """Encode an image file to base64."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [52]:
session = boto3.Session()
runtime = session.client('sagemaker-runtime')


payload = {
    "image": encode_image_to_base64(image_path),
    "confidence_threshold": 0.3,
    "visualize": False
}

# Convert the payload dictionary to a JSON string
body = json.dumps(payload)

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=body  # Pass the JSON string, not the Python dictionary
)

result = json.loads(response['Body'].read().decode())
print(json.dumps(result, indent=2))


[05/12/25 06:22:48] INFO     Found credentials from IAM Role:                                   ]8;id=810080;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=836347;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

{
  "detections": [
    {
      "box": {
        "x1": 33.38,
        "y1": 0.65,
        "x2": 385.45,
        "y2": 288.65
      },
      "confidence": 0.9489,
      "class_id": 0,
      "class": "person"
    },
    {
      "box": {
        "x1": 152.27,
        "y1": 123.86,
        "x2": 219.47,
        "y2": 288.12
      },
      "confidence": 0.5079,
      "class_id": 27,
      "class": "tie"
    }
  ],
  "inference_time": 2.2881,
  "image_width": 390,
  "image_height": 288
}


In [53]:
import boto3

sagemaker_client = boto3.client('sagemaker')
response = sagemaker_client.list_endpoints()

# Print endpoint names
for endpoint in response['Endpoints']:
    print(endpoint['EndpointName'])

deployment-rtdetr-gpu
